In [1]:
! ls ../data/FCC-CLD/gun/*.root    # must confirm which CLD version?

../data/FCC-CLD/gun/out_reco_edm4hep_REC.edm4hep.root
../data/FCC-CLD/gun/out_sim_edm4hep.root


In [292]:
import math

import numpy as np
import uproot
import awkward as ak
import vector

vector.register_awkward()

import particle
from particle import Particle

import networkx as nx
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
# pio.renderers.default = "iframe"
pio.renderers.default = "browser"

In [304]:
# PATH = "../data/FCC-CLD/gun/out_reco_edm4hep_REC.edm4hep.root"
PATH = "../data/FCC-CLD/ttbar/reco_p8_ee_tt_ecm365_0.root"
f = uproot.open(PATH)

f.keys()

['events;2',
 'events;1',
 'configuration_metadata;1',
 'metadata;1',
 'podio_metadata;1']

In [305]:
ev = f["events"]
metadata = f["metadata"]

print(f"EDM4HEP version: {f['podio_metadata/edm4hep___Version'].array()}")
print(f"PodioBuild version: {f['podio_metadata/PodioBuildVersion'].array()}")
print(f"# of events: {ev.num_entries}")

EDM4HEP version: [{major: 0, minor: 99, patch: 1}]
PodioBuild version: [{major: 1, minor: 1, patch: 99}]
# of events: 100


In [306]:
def get_collectionIDs(f, iev):
    # build collectionID -> collectionName mapping
    collectionIDs = {
        k: v
        for k, v in zip(
            f.get("podio_metadata").arrays("events___idTable/m_names")["events___idTable/m_names"][0],
            f.get("podio_metadata").arrays("events___idTable/m_collectionIDs")["events___idTable/m_collectionIDs"][0],
        )
    }

    collectionIDs_reverse = {cid: name for name, cid in collectionIDs.items()}
    
    return collectionIDs_reverse    

In [307]:
# sanity check which side is hits vs MCParticles 

collectionIDs_reverse = get_collectionIDs(f, iev)

k = 0
print("from collID:", ev["_CalohitMCTruthLink_from/_CalohitMCTruthLink_from.collectionID"].array()[iev][k])
print("to   collID:", ev["_CalohitMCTruthLink_to/_CalohitMCTruthLink_to.collectionID"].array()[iev][k])

print("from coll name:", collectionIDs_reverse[int(ev["_CalohitMCTruthLink_from/_CalohitMCTruthLink_from.collectionID"].array()[iev][k])])
print("to   coll name:", collectionIDs_reverse[int(ev["_CalohitMCTruthLink_to/_CalohitMCTruthLink_to.collectionID"].array()[iev][k])])

from collID: 1265367388
to   collID: 2714477136
from coll name: ECALBarrel
to   coll name: MCParticles


In [310]:
def build_relation_lists_from_offsets(
    begin,
    end,
    relation_index,
    n_items,
    name="relation",
):
    """
    Build per-item relation lists from EDM-style begin/end offsets (useful to trace back parent-daughter relations).

    Parameters
    ----------
    begin : array-like (n_items)
        Begin offsets into the flat relation index buffer.
    end : array-like (n_items)
        End offsets into the flat relation index buffer.
    relation_index : array-like (n_links)
        Flat buffer containing indices of related items.
    n_items : int
        Number of items the relations refer to (e.g. number of MCParticles).
    name : str
        Name of the relation (used for error messages).

    Returns
    -------
    relations : awkward.Array
        Jagged array of length n_items, where relations[i] is
        a list of indices related to item i.
    """

    # Convert to plain numpy for predictable slicing / checks
    begin = np.asarray(begin)
    end = np.asarray(end)

    if begin.shape != end.shape:
        raise ValueError(
            f"{name}: begin/end shape mismatch: {begin.shape} vs {end.shape}"
        )

    if len(begin) != n_items:
        raise ValueError(
            f"{name}: expected begin/end of length {n_items}, got {len(begin)}"
        )

    if np.any(end < begin):
        raise ValueError(f"{name}: found end < begin (corrupt offsets)")

    # Number of relations per item
    counts = end - begin

    # Extract each relation slice explicitly (very clear semantics)
    slices = [
        relation_index[b:e]
        for b, e in zip(begin.tolist(), end.tolist())
    ]

    # Flatten then unflatten to guarantee correct jagged structure
    flat = ak.concatenate(slices, axis=0) if slices else relation_index[:0]

    if len(flat) != int(counts.sum()):
        raise ValueError(
            f"{name}: inconsistent offsets: "
            f"sum(end-begin)={int(counts.sum())}, "
            f"but flattened relation buffer has len={len(flat)}"
        )

    relations = ak.unflatten(flat, counts)

    # Final safety: indices must point to valid items
    if len(flat) > 0:
        min_idx = int(ak.min(flat))
        max_idx = int(ak.max(flat))
        if min_idx < 0 or max_idx >= n_items:
            raise ValueError(
                f"{name}: relation index out of range "
                f"(min={min_idx}, max={max_idx}, n_items={n_items})"
            )

    return relations

def get_mc_particles(ev, iev):
    """
    Build MC particle record for a single event.
    Adds:
      - mc['parents']   : jagged list of parent indices per particle
      - mc['daughters'] : jagged list of daughter indices per particle
    and derived pt/eta/phi/energy.
    """

    # --- base per-particle fields (for ONE event) ---
    mc = ak.zip({
        "pdg": ev["MCParticles.PDG"].array()[iev],
        "genstatus": ev["MCParticles.generatorStatus"].array()[iev],
        "simstatus": ev["MCParticles.simulatorStatus"].array()[iev],
        "charge": ev["MCParticles.charge"].array()[iev],
        "time": ev["MCParticles.time"].array()[iev],

        "px": ev["MCParticles.momentum.x"].array()[iev],
        "py": ev["MCParticles.momentum.y"].array()[iev],
        "pz": ev["MCParticles.momentum.z"].array()[iev],
        "mass": ev["MCParticles.mass"].array()[iev],

        "vx": ev["MCParticles.vertex.x"].array()[iev],
        "vy": ev["MCParticles.vertex.y"].array()[iev],
        "vz": ev["MCParticles.vertex.z"].array()[iev],

        "endx": ev["MCParticles.endpoint.x"].array()[iev],
        "endy": ev["MCParticles.endpoint.y"].array()[iev],
        "endz": ev["MCParticles.endpoint.z"].array()[iev],

        "endpx": ev["MCParticles.momentumAtEndpoint.x"].array()[iev],
        "endpy": ev["MCParticles.momentumAtEndpoint.y"].array()[iev],
        "endpz": ev["MCParticles.momentumAtEndpoint.z"].array()[iev],

        "spinx": ev["MCParticles.spin.x"].array()[iev],
        "spiny": ev["MCParticles.spin.y"].array()[iev],
        "spinz": ev["MCParticles.spin.z"].array()[iev],

        "color_a": ev["MCParticles.colorFlow.a"].array()[iev],
        "color_b": ev["MCParticles.colorFlow.b"].array()[iev],
    })

    nMC = len(mc)

    # --- relation pointers (ONE event) ---
    pb = ev["MCParticles.parents_begin"].array()[iev]
    pe = ev["MCParticles.parents_end"].array()[iev]
    db = ev["MCParticles.daughters_begin"].array()[iev]
    de = ev["MCParticles.daughters_end"].array()[iev]

    parents_index = ev["_MCParticles_parents/_MCParticles_parents.index"].array()[iev]
    daughters_index = ev["_MCParticles_daughters/_MCParticles_daughters.index"].array()[iev]

    parents = build_relation_lists_from_offsets(
        pb, pe, parents_index, nMC, name="parents"
    )

    daughters = build_relation_lists_from_offsets(
        db, de, daughters_index, nMC, name="daughters"
    )

    # Attach (this avoids ak.zip broadcasting issues)
    mc = ak.with_field(mc, parents, "parents")
    mc = ak.with_field(mc, daughters, "daughters")

    # --- derived kinematics via vector ---
    p4 = ak.zip({"px": mc.px, "py": mc.py, "pz": mc.pz, "mass": mc.mass}, with_name="Momentum4D")
    mc = ak.with_field(mc, p4.pt, "pt")
    mc = ak.with_field(mc, p4.eta, "eta")
    mc = ak.with_field(mc, p4.phi, "phi")
    mc = ak.with_field(mc, p4.energy, "energy")

    return mc

In [311]:
iev = 0

mc = get_mc_particles(ev, iev)

nMC = len(mc["pdg"])
nMC

1103

In [312]:
for iMC in range(nMC):
    print(f"{iMC}, GenStatus: {mc['genstatus'][iMC]}, PID: {abs(mc['pdg'][iMC])}, pt: {mc['pt'][iMC].round(3)} - (SimStatus: {mc['simstatus'][iMC]})")

0, GenStatus: 4, PID: 11, pt: 2.738 - (SimStatus: 0)
1, GenStatus: 41, PID: 11, pt: 2.738 - (SimStatus: 268435456)
2, GenStatus: 1, PID: 22, pt: 0.0 - (SimStatus: 301989888)
3, GenStatus: 4, PID: 11, pt: 2.738 - (SimStatus: 0)
4, GenStatus: 42, PID: 11, pt: 2.738 - (SimStatus: 268435456)
5, GenStatus: 1, PID: 22, pt: 0.0 - (SimStatus: 352321536)
6, GenStatus: 21, PID: 11, pt: 2.738 - (SimStatus: 268435456)
7, GenStatus: 1, PID: 22, pt: 0.007 - (SimStatus: 352321536)
8, GenStatus: 21, PID: 11, pt: 2.738 - (SimStatus: 268435456)
9, GenStatus: 22, PID: 6, pt: 38.966 - (SimStatus: 268435456)
10, GenStatus: 22, PID: 6, pt: 38.222 - (SimStatus: 268435456)
11, GenStatus: 44, PID: 6, pt: 38.962 - (SimStatus: 268435456)
12, GenStatus: 44, PID: 6, pt: 38.224 - (SimStatus: 268435456)
13, GenStatus: 22, PID: 24, pt: 53.832 - (SimStatus: 268435456)
14, GenStatus: 23, PID: 5, pt: 62.909 - (SimStatus: 268435456)
15, GenStatus: 22, PID: 24, pt: 57.42 - (SimStatus: 268435456)
16, GenStatus: 23, PID: 5,

1087, GenStatus: 0, PID: 2212, pt: 0.055 - (SimStatus: 1493172224)
1088, GenStatus: 0, PID: 1000060120, pt: 0.101 - (SimStatus: 1493172224)
1089, GenStatus: 0, PID: 22, pt: 0.002 - (SimStatus: 1493172224)
1090, GenStatus: 0, PID: 22, pt: 0.001 - (SimStatus: 1493172224)
1091, GenStatus: 0, PID: 11, pt: 0.001 - (SimStatus: 1493172224)
1092, GenStatus: 0, PID: 11, pt: 0.002 - (SimStatus: 1493172224)
1093, GenStatus: 0, PID: 11, pt: 0.001 - (SimStatus: 1493172224)
1094, GenStatus: 0, PID: 11, pt: 0.003 - (SimStatus: 1493172224)
1095, GenStatus: 0, PID: 2212, pt: 0.06 - (SimStatus: 1493172224)
1096, GenStatus: 0, PID: 22, pt: 0.002 - (SimStatus: 1426063360)
1097, GenStatus: 0, PID: 22, pt: 0.002 - (SimStatus: 1426063360)
1098, GenStatus: 0, PID: 11, pt: 0.002 - (SimStatus: 1493172224)
1099, GenStatus: 0, PID: 11, pt: 0.001 - (SimStatus: 1493172224)
1100, GenStatus: 0, PID: 11, pt: 0.001 - (SimStatus: 1493172224)
1101, GenStatus: 0, PID: 2212, pt: 0.149 - (SimStatus: 1493172224)
1102, GenSta

In [313]:
for iMC in range(nMC):
    print(f"{iMC}, parents: {mc['parents'][iMC]}, daughters: {mc['daughters'][iMC]}")

0, parents: [], daughters: [1, 2]
1, parents: [0], daughters: [6, 7]
2, parents: [0], daughters: []
3, parents: [], daughters: [4, 5]
4, parents: [3], daughters: [8]
5, parents: [3], daughters: []
6, parents: [1], daughters: [9, 10]
7, parents: [1], daughters: []
8, parents: [4], daughters: [9, 10]
9, parents: [6, 8], daughters: [11]
10, parents: [6, 8], daughters: [12]
11, parents: [9], daughters: [13, 14]
12, parents: [10], daughters: [15, 16]
13, parents: [11], daughters: [17]
14, parents: [11], daughters: [18, 19]
15, parents: [12], daughters: [20]
16, parents: [12], daughters: [21, 22]
17, parents: [13], daughters: [23, 24]
18, parents: [14], daughters: [25, 26]
19, parents: [14], daughters: [27]
20, parents: [15], daughters: [28, 29]
21, parents: [16], daughters: [30]
22, parents: [16], daughters: [31, 32]
23, parents: [17], daughters: [33]
24, parents: [17], daughters: [34, 35]
25, parents: [18], daughters: [36]
26, parents: [18], daughters: [37]
27, parents: [19], daughters: [3

# Decay tress

In [361]:
def draw_mc_graph_interactive(
    mc,
    min_energy=None,                 # existing: prune low-energy subtrees
    # --- NEW: select a subgraph around "seed" nodes ---
    seed_idx=None,                   # int or list[int]
    seed_pdg=None,                   # int or list[int]
    seed_min_pt=None,
    seed_min_energy=None,
    seed_genstatus=None,             # int or list[int]
    show_ancestors=True,
    show_descendants=True,
):
    g = mc_to_graph(mc)

    n = len(mc)
    pt = np.array(mc["pt"], dtype=float)
    E  = np.array(mc["energy"], dtype=float)
    pdg = np.array(mc["pdg"], dtype=int)
    gs  = np.array(mc["genstatus"], dtype=int)

    # -----------------------------
    # 1) Optional: min_energy subtree prune (your existing behavior)
    # -----------------------------
    if min_energy is not None:
        roots = [u for u in g.nodes() if g.in_degree(u) == 0]
        keep_prune = set()
        stack = list(roots)

        while stack:
            u = stack.pop()
            if E[u] < float(min_energy):
                continue
            keep_prune.add(u)
            for v in g.successors(u):
                if all((p in keep_prune) for p in g.predecessors(v)):
                    stack.append(v)

        g = g.subgraph(keep_prune).copy()

    # -----------------------------
    # 2) Build seed mask (which nodes define the "region of interest")
    # -----------------------------
    seed_mask = np.ones(n, dtype=bool)

    if seed_idx is not None:
        idx_mask = np.zeros(n, dtype=bool)
        if isinstance(seed_idx, (list, tuple, set, np.ndarray)):
            idx_mask[list(seed_idx)] = True
        else:
            idx_mask[int(seed_idx)] = True
        seed_mask &= idx_mask

    if seed_pdg is not None:
        if isinstance(seed_pdg, (list, tuple, set, np.ndarray)):
            seed_mask &= np.isin(pdg, list(seed_pdg))
        else:
            seed_mask &= (pdg == int(seed_pdg))

    if seed_min_pt is not None:
        seed_mask &= (pt >= float(seed_min_pt))

    if seed_min_energy is not None:
        seed_mask &= (E >= float(seed_min_energy))

    if seed_genstatus is not None:
        if isinstance(seed_genstatus, (list, tuple, set, np.ndarray)):
            seed_mask &= np.isin(gs, list(seed_genstatus))
        else:
            seed_mask &= (gs == int(seed_genstatus))

    seeds = [i for i in range(n) if seed_mask[i] and i in g]  # ensure node exists after prune

    # If no seed selection provided, plot whole (possibly pruned) tree
    any_seed_filter = any(v is not None for v in [seed_idx, seed_pdg, seed_min_pt, seed_min_energy, seed_genstatus])
    if any_seed_filter and len(seeds) == 0:
        print("No seed nodes matched the selection (after pruning).")
        return

    # -----------------------------
    # 3) Expand seeds to include ancestors/descendants
    # -----------------------------
    if any_seed_filter:
        keep = set(seeds)

        if show_ancestors:
            stack = list(seeds)
            while stack:
                u = stack.pop()
                for p in g.predecessors(u):
                    if p not in keep:
                        keep.add(p)
                        stack.append(p)

        if show_descendants:
            stack = list(seeds)
            while stack:
                u = stack.pop()
                for v in g.successors(u):
                    if v not in keep:
                        keep.add(v)
                        stack.append(v)

        g = g.subgraph(keep).copy()

    # -----------------------------
    # 4) Layout + plotting (your existing code, mostly unchanged)
    # -----------------------------
    pos = nx.nx_agraph.graphviz_layout(g, prog="dot")

    edge_x, edge_y = [], []
    for u, v in g.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        mode="lines",
        line=dict(width=1),
        hoverinfo="skip",
        opacity=0.25,
    )

    nodes = list(g.nodes())
    x = np.array([pos[i][0] for i in nodes], dtype=float)
    y = np.array([pos[i][1] for i in nodes], dtype=float)

    node_size = np.clip(30 * (pt[nodes] + 1e-3) ** 0.6, 8, 70)
    node_color = [color_node(int(mc["genstatus"][i])) for i in nodes]

    hovertext = []
    for i in nodes:
        s = (
            f"{Particle.from_pdgid(int(mc['pdg'][i])).name}<br>"
            f"idx: {i}<br>"
            f"genstatus: {int(mc['genstatus'][i])}<br>"
            f"charge: {float(mc['charge'][i]):.3g}<br>"
            f"E: {float(mc['energy'][i]):.4g}<br>"
            f"pT: {float(mc['pt'][i]):.4g}<br>"
            f"eta: {float(mc['eta'][i]):.4g}<br>"
            f"phi: {float(mc['phi'][i]):.4g}"
        )
                    
        parents = list(mc["parents"][i])
        if len(parents) == 0:
            s += "<br>ΔR(parent): —"
        else:
            eta_i = float(mc["eta"][i])
            phi_i = float(mc["phi"][i])
            drs = []
            lines = []
            for p in parents:
                p = int(p)
                if p not in g:  # if parent not in subgraph, skip listing
                    continue
                d_eta = eta_i - float(mc["eta"][p])
                d_phi = phi_i - float(mc["phi"][p])
                d_phi = (d_phi + np.pi) % (2 * np.pi) - np.pi
                dr = float(np.hypot(d_eta, d_phi))
                drs.append(dr)
                lines.append(f"p={p} (PDG {int(mc['pdg'][p])}): ΔR={dr:.3g}")
            if len(drs) == 0:
                s += "<br>ΔR(parent): —"
            else:
                s += f"<br>ΔR(parent) min: {min(drs):.3g}"
                s += "<br>" + "<br>".join(lines)

        hovertext.append(s)

    text = [f"{int(mc['pdg'][i])}" for i in nodes]

    node_trace = go.Scatter(
        x=x, y=y,
        mode="markers+text",
        text=text,
        textposition="middle center",
        hovertext=hovertext,
        hoverinfo="text",
        # THIS controls hover text appearance
        hoverlabel=dict(
            font_size=20,        # try 14–20
            font_family="Arial",
            bgcolor="white",     # optional
            font_color="black",  # optional
            bordercolor="black", # optional
        ),                            
        textfont=dict(size=20, color="black", family="Arial"),
        marker=dict(size=node_size, color=node_color, line=dict(width=1, color="white"), opacity=0.65),
    )

    fig = go.Figure(data=[edge_trace, node_trace])
    fig.update_layout(
        title="MC graph (hover for details)",
        showlegend=False,
        hovermode="closest",
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        margin=dict(l=20, r=20, t=40, b=20),
    )
    fig.show()


In [363]:
# draw_mc_graph_interactive(mc, min_energy=10)    # msks all partricles with E<X (not sure how useful this is)

# draw_mc_graph_interactive(mc, seed_genstatus=1, show_descendants=False, show_ancestors=True)
# draw_mc_graph_interactive(mc, seed_pdg=[13, -13], seed_min_pt=1)

draw_mc_graph_interactive(mc, seed_idx=103)

# Color hits by assigned gen particle

In [419]:
import colorsys

def _idx_to_color(i, n):
    # distinct-ish colors around HSV wheel
    h = (i / max(n, 1)) % 1.0
    r, g, b = colorsys.hsv_to_rgb(h, 0.65, 0.95)
    return f"rgb({int(255*r)},{int(255*g)},{int(255*b)})"

calo_cols = [
    "ECALBarrel",
    "ECALEndcap",
    "HCALBarrel",
    "HCALEndcap",
    "HCALOther",
    "MUON",
]

def plot_calo_hits_mc_linked_3d(
    ev, iev, mc,
    clip_mm=8000, log_size=True,
    only_best_weight=True,
    # --- filters on the linked MC particle ---
    mc_min_pt=None,
    mc_min_energy=None,
    mc_pdg=None,    
    mc_genstatus=None,
    mc_idx=None,    
    require_linked=True,    
):
    
    # --- allowed MC indices mask ---
    nmc = len(mc)
    allowed = np.ones(nmc, dtype=bool)

    if mc_min_pt is not None:
        allowed &= (np.asarray(mc["pt"], dtype=float) >= float(mc_min_pt))
    if mc_min_energy is not None:
        allowed &= (np.asarray(mc["energy"], dtype=float) >= float(mc_min_energy))
    if mc_pdg is not None:
        pdg = np.asarray(mc["pdg"], dtype=int)
        if isinstance(mc_pdg, (list, tuple, set, np.ndarray)):
            allowed &= np.isin(pdg, list(mc_pdg))
        else:
            allowed &= (pdg == int(mc_pdg))
    if mc_idx is not None:
        idx_mask = np.zeros(nmc, dtype=bool)
        if isinstance(mc_idx, (list, tuple, set, np.ndarray)):
            idx_mask[list(mc_idx)] = True
        else:
            idx_mask[int(mc_idx)] = True
        allowed &= idx_mask            
    if mc_genstatus is not None:
        gs = np.asarray(mc["genstatus"], dtype=int)
        if isinstance(mc_genstatus, (list, tuple, set, np.ndarray)):
            allowed &= np.isin(gs, list(mc_genstatus))
        else:
            allowed &= (gs == int(mc_genstatus))

    # ---------- build hit -> MC mapping for each calo collection ----------
    hit_to_mc = {}
    hit_best_w = {}

    for name in calo_cols:
        if name not in ev.keys():
            continue
        nh = len(ev[f"{name}.energy"].array()[iev])
        hit_to_mc[name] = -1 * np.ones(nh, dtype=np.int32)
        hit_best_w[name] = -1.0 * np.ones(nh, dtype=np.float32)

    # relation arrays (one entry per link)
    w = np.asarray(ev["CalohitMCTruthLink/CalohitMCTruthLink.weight"].array()[iev])

    fr_idx  = np.asarray(ev["_CalohitMCTruthLink_from/_CalohitMCTruthLink_from.index"].array()[iev])
    fr_coll = np.asarray(ev["_CalohitMCTruthLink_from/_CalohitMCTruthLink_from.collectionID"].array()[iev])

    to_idx  = np.asarray(ev["_CalohitMCTruthLink_to/_CalohitMCTruthLink_to.index"].array()[iev])
    to_coll = np.asarray(ev["_CalohitMCTruthLink_to/_CalohitMCTruthLink_to.collectionID"].array()[iev])

    for k in range(len(w)):
        from_name = collectionIDs_reverse.get(int(fr_coll[k]), None)
        to_name   = collectionIDs_reverse.get(int(to_coll[k]), None)

        # We want from = calo hits, to = MCParticles
        if from_name not in hit_to_mc:
            continue
        if to_name != "MCParticles":
            continue

        h = int(fr_idx[k])      # hit index within that hit collection
        m = int(to_idx[k])      # MC particle index within MCParticles

        if h < 0 or h >= len(hit_to_mc[from_name]):
            continue

        if only_best_weight:
            if w[k] > hit_best_w[from_name][h]:
                hit_best_w[from_name][h] = w[k]
                hit_to_mc[from_name][h] = m
        else:
            # if you want "any link" just overwrite
            hit_to_mc[from_name][h] = m

    import colorsys

    def _idx_to_color(idx):
        # Deterministic color from MC index using golden-ratio spacing in hue
        h = (idx * 0.61803398875) % 1.0
        r, g, b = colorsys.hsv_to_rgb(h, 0.65, 0.95)
        return f"rgb({int(255*r)},{int(255*g)},{int(255*b)})"

    # Collect all linked MC indices across *all* calo collections
    all_linked = set()
    for name in calo_cols:
        if name in hit_to_mc:
            link = hit_to_mc[name]
            all_linked.update(map(int, np.unique(link[link >= 0])))

    color_map = {m: _idx_to_color(m) for m in all_linked}

    # ---------- plot ----------
    traces = []

    # build a single global colorscale for mc indices
    # plotly will color by numeric value; -1 will be a special "unlinked" color (we handle separately)
    for name in calo_cols:
        if name not in hit_to_mc:
            continue

        x = np.asarray(ev[f"{name}.position.x"].array()[iev])
        y = np.asarray(ev[f"{name}.position.y"].array()[iev])
        z = np.asarray(ev[f"{name}.position.z"].array()[iev])
        e = np.asarray(ev[f"{name}.energy"].array()[iev])
        link = hit_to_mc[name]

        m = np.isfinite(x) & np.isfinite(y) & np.isfinite(z) & np.isfinite(e) & (e > 0)
        
        # require linked hits (optional)
        if require_linked:
            m &= (link >= 0)

        # apply MC filters via allowed mask
        # keep only hits whose linked MC idx passes selection
        linked_ok = np.zeros_like(m, dtype=bool)
        ok_link = link >= 0
        linked_ok[ok_link] = allowed[link[ok_link]]
        m &= linked_ok
        
        x, y, z, e, link = x[m], y[m], z[m], e[m], link[m]

        if len(e) == 0:
            continue

        # marker size ~ energy (same style as you)
        if log_size:
            size = np.clip(2 + 4 * np.log10(e / np.max(e) + 1e-6), 1, 10)
        else:
            size = np.clip(1 + 30 * (e / np.max(e)), 1, 12)
        size = size + 5

        # Hover text: include MC info if linked
        hovertext = []
        for ih in range(len(e)):
            mi = int(link[ih])
            if mi >= 0:
                hovertext.append(
                    f"{Particle.from_pdgid(mc['pdg'][mi]).name}<br>"
                    f"idx={mi}<br>"
                    f"genstatus={float(mc['genstatus'][mi]):.3g}<br>"
                    f"charge={float(mc['charge'][mi]):.3g}<br>"
                    f"E={float(mc['energy'][mi]):.3g} GeV<br>"
                    f"hit E={float(e[ih]):.3g}<br>"
                    f"{name}<br>"                    
#                     f"pT={float(mc['pt'][mi]):.3g}, eta={float(mc['eta'][mi]):.3g}, phi={float(mc['phi'][mi]):.3g}"
                )
            else:
                hovertext.append(f"{name}<br>hit E={float(e[ih]):.3g}<br>MC idx=—")

        # Color: by linked MC index
        # For unlinked (-1), set a constant neutral color by splitting into 2 traces:
        linked_mask = link >= 0
        unlinked_mask = ~linked_mask

        hit_colors_linked = [color_map[int(m)] for m in link[linked_mask]]

        if np.any(unlinked_mask):
            traces.append(
                go.Scatter3d(
                    x=np.clip(x[unlinked_mask], -clip_mm, clip_mm),
                    y=np.clip(y[unlinked_mask], -clip_mm, clip_mm),
                    z=np.clip(z[unlinked_mask], -clip_mm, clip_mm),
                    mode="markers",
                    marker=dict(size=size[unlinked_mask], color="lightgray", opacity=0.6),
                    name=f"{name} (unlinked)",
                    hovertext=np.array(hovertext, dtype=object)[unlinked_mask],
                    hoverinfo="text",
                )
            )

        if np.any(linked_mask):
            traces.append(
                go.Scatter3d(
                    x=np.clip(x[linked_mask], -clip_mm, clip_mm),
                    y=np.clip(y[linked_mask], -clip_mm, clip_mm),
                    z=np.clip(z[linked_mask], -clip_mm, clip_mm),
                    mode="markers",
                    marker=dict(
                        size=size[linked_mask],
                        color=hit_colors_linked,  # <-- categorical colors, masked correctly
                        opacity=1.0,
                    ),
                    name=f"{name} (linked)",
                    hovertext=np.array(hovertext, dtype=object)[linked_mask],
                    hoverinfo="text",
                    hoverlabel=dict(
                        font_size=20,
                        font_family="Arial",
                        bgcolor="white",
                        font_color="black",
                        bordercolor="black",
                    ),
                )
            )

    fig = go.Figure(data=traces)
    fig.update_layout(
        title=f"Calo hits colored by linked MC index (event {iev})",
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="z",
            aspectmode="data",
            camera=dict(
                eye=dict(x=0.0, y=0.0, z=2.5),
                up=dict(x=0.0, y=1.0, z=0.0),
            ),
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        legend=dict(itemsizing="constant"),
    )
    fig.show()

In [406]:
# config which applies a mask on the hits belonging to the MC particle
# use None to apply no mask
mc_sel = dict(
    mc_genstatus=1,
    mc_pdg=None,
    mc_min_energy=None,
    mc_min_pt=None,
    mc_idx=None,
)

plot_calo_hits_mc_linked_3d(ev, iev, mc, **mc_sel)

# Playground

In [421]:
# config which applies a mask on the hits belonging to the MC particle
# use None to apply no mask
mc_sel = dict(
    mc_genstatus=None,
    mc_pdg=None,
    mc_min_energy=5,
    mc_min_pt=None,
#     mc_idx=[271],
)

plot_calo_hits_mc_linked_3d(ev, iev, mc, **mc_sel)

In [424]:
draw_mc_graph_interactive(mc)#, seed_idx=[79])